In [1]:
# v0.10
# Dependencies: OpenSSL installed (to handle https/SSL), requests module
# Notes: The requests module handles most of the heavy lifting
#       This will loop through all results in a query, 50 at a time,
#       saving individual JSON files to the same directory as the script

import requests
from requests.auth import HTTPBasicAuth
from datetime import datetime  # used to name json files
from time import sleep
import json

In [14]:
# put all states (base64) into a list, 10 per element. 
Location_filter = ["Location eq 'VVM' or Location eq 'VVMtQUw' or Location eq 'VVMtQUs' or Location eq 'VVMtQVo' or Location eq 'VVMtQVI' or Location eq 'VVMtQ0E' or Location eq 'VVMtQ08' or Location eq 'VVMtQ1Q' or Location eq 'VVMtREM' or Location eq 'VVMtRkw'","Location eq 'VVMtR0E' or Location eq 'VVMtSUQ' or Location eq 'VVMtSUw' or Location eq 'VVMtSU4' or Location eq 'VVMtSUE' or Location eq 'VVMtS1Mg' or Location eq 'VVMtS1k' or Location eq 'VVMtTEE' or Location eq 'VVMtTUU' or Location eq 'VVMtTUQg'", "Location eq 'VVMtTUE' or Location eq 'VVMtTUk' or Location eq 'VVMtTU4g' or Location eq 'VVMtTVM' or Location eq 'VVMtTU8' or Location eq 'VVMtTkU' or Location eq 'VVMtTlY' or Location eq 'VVMtTko' or Location eq 'VVMtTk0' or Location eq 'VVMtTlk'", "Location eq 'VVMtTkM' or Location eq 'VVMtTkQ' or Location eq 'VVMtT0g' or Location eq 'VVMtT0s' or Location eq 'VVMtT1Ig' or Location eq 'VVMtUEE' or Location eq 'VVMtUkk' or Location eq 'VVMtU0M' or Location eq 'VVMtVE4' or Location eq 'VVMtVFg'", "Location eq 'VVMtVVQ' or Location eq 'VVMtVkE' or Location eq 'VVMtV0E' or Location eq 'VVMtV1Y' or Location eq 'VVMtV0k' or Location eq 'VVMtV1kg'"]
#5 Elements in total for 46 states.
query = 'recession and date > 2006 and date < 2010' 
#Place entire query string inside these quotes. (e.g. "'Edwin Willers'")

["Location eq 'VVM' or Location eq 'VVMtQUw' or Location eq 'VVMtQUs' or Location eq 'VVMtQVo' or Location eq 'VVMtQVI' or Location eq 'VVMtQ0E' or Location eq 'VVMtQ08' or Location eq 'VVMtQ1Q' or Location eq 'VVMtREM' or Location eq 'VVMtRkw'", "Location eq 'VVMtR0E' or Location eq 'VVMtSUQ' or Location eq 'VVMtSUw' or Location eq 'VVMtSU4' or Location eq 'VVMtSUE' or Location eq 'VVMtS1Mg' or Location eq 'VVMtS1k' or Location eq 'VVMtTEE' or Location eq 'VVMtTUU' or Location eq 'VVMtTUQg'", "Location eq 'VVMtTUE' or Location eq 'VVMtTUk' or Location eq 'VVMtTU4g' or Location eq 'VVMtTVM' or Location eq 'VVMtTU8' or Location eq 'VVMtTkU' or Location eq 'VVMtTlY' or Location eq 'VVMtTko' or Location eq 'VVMtTk0' or Location eq 'VVMtTlk'", "Location eq 'VVMtTkM' or Location eq 'VVMtTkQ' or Location eq 'VVMtT0g' or Location eq 'VVMtT0s' or Location eq 'VVMtT1Ig' or Location eq 'VVMtUEE' or Location eq 'VVMtUkk' or Location eq 'VVMtU0M' or Location eq 'VVMtVE4' or Location eq 'VVMtVFg'",

In [ ]:
client_id = 'F8PT8FS9PXXRBR78RMZFJ25J8T526P'  # real Client ID
secret = 'FPMJP84Q772GWHZ727M2STGPQ84MJ69MNPKS983H'  #  real Secret

In [ ]:
############# Begin Function Definitions #############

def get_token(client_id, secret):
    """Gets Authorizaton token to use in other requests."""
    auth_url = 'https://auth-api.lexisnexis.com/oauth/v2/token'
    payload = ('grant_type=client_credentials&scope=http%3a%2f%2f'
                'oauth.lexisnexis.com%2fall')
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    r = requests.post(
            auth_url,
            auth=HTTPBasicAuth(client_id, secret),
            headers=headers,
            data=payload)
    json_data = r.json()
    return json_data['access_token']


def build_url(content='News', query='', skip=0, expand='Document', top=50, filter=filter):
    """Builds the URL part of the request to Web Services API."""
    if filter != None:  # Filter is an optional parameter
        api_url = ('https://services-api.lexisnexis.com/v1/' + content +
                    '?$expand=' + expand + '&$search=' + query +
                    '&$skip=' + str(skip) + '&$top=' + str(top) +
                    '&$filter=' + filter)
    else:
        api_url = ('https://services-api.lexisnexis.com/v1/' + content +
                    '?$expand=' + expand + '&$search=' + query +
                    '&$skip=' + str(skip) + '&$top=' + str(top))
    return api_url


def build_header(token):
    """Builds the headers part of the request to Web Services API."""
    headers = {'Accept': 'application/json;odata.metadata=minimal',
                'Connection': 'Keep-Alive',
                'Host': 'services-api.lexisnexis.com'}
    headers['Authorization'] = 'Bearer ' + token
    return headers


def get_result_count(json_data):
    """Gets the number of results from @odata.count in the response"""
    return json_data['@odata.count']


def time_now():
    """Gets current time to the second."""
    now = datetime.now()
    return now.strftime('%Y-%m-%d-%H%M%S')

############# End Function Defnitions #############

In [15]:
############# Begin business logic #############

token = get_token(client_id, secret)  # 1 token will work for multiple requests
request_headers = build_header(token)
skip_value = 0  # Sets starting skip
top = 50  # Adjusts the number of results to return

#Place entire query string inside these quotes. (e.g. "SearchType eq LexisNexis.ServicesApi.SearchType'Boolean' and PublicationType eq 'TmV3c3BhcGVycw' and Language eq LexisNexis.ServicesApi.Language'English'") #### filter key is base64. e.g. US-OH is VVMtT0g
# loops through all states, 10 states at a time, to avoid reaching node limit
for i in range(len(Location_filter)):
        filter = "SearchType eq LexisNexis.ServicesApi.SearchType'Boolean' and (PublicationType eq 'SW5kdXN0cnkgVHJhZGUgUHJlc3M' or PublicationType eq 'TmV3c3dpcmVzICYgUHJlc3MgUmVsZWFzZXM'or PublicationType eq 'TmV3c3BhcGVycw' or PublicationType eq 'TWFnYXppbmVzICYgSm91cm5hbHM') and Language eq LexisNexis.ServicesApi.Language'English' and GroupDuplicates eq LexisNexis.ServicesApi.GroupDuplicates'ModerateSimilarity' and (" + Location_filter[i] +")"
        while True:
            request_url = build_url(content='News', query=query, skip=skip_value, expand='Document', top=top, filter=filter)  # Filter is set to filter=None here. Change to filter=filter to use the filter specified above
            r = requests.get(request_url, headers=request_headers)
            
            with open(str(time_now()) + '.json', 'w') as f_out:  # Creates a file with the current time as the file name.
                f_out.write(r.text)
                
            skip_value = (skip_value + top)
            json_data = r.json()
            if skip_value > get_result_count(json_data):  # Check to see if all the results have been looped through
                break
            sleep(12)  # Limit 5 requests per minute (every 12 seconds)
        

5
SearchType eq LexisNexis.ServicesApi.SearchType'Boolean' and (PublicationType eq 'SW5kdXN0cnkgVHJhZGUgUHJlc3M' or PublicationType eq 'TmV3c3dpcmVzICYgUHJlc3MgUmVsZWFzZXM'or PublicationType eq 'TmV3c3BhcGVycw' or PublicationType eq 'TWFnYXppbmVzICYgSm91cm5hbHM') and Language eq LexisNexis.ServicesApi.Language'English' and GroupDuplicates eq LexisNexis.ServicesApi.GroupDuplicates'ModerateSimilarity' and (Location eq 'VVM' or Location eq 'VVMtQUw' or Location eq 'VVMtQUs' or Location eq 'VVMtQVo' or Location eq 'VVMtQVI' or Location eq 'VVMtQ0E' or Location eq 'VVMtQ08' or Location eq 'VVMtQ1Q' or Location eq 'VVMtREM' or Location eq 'VVMtRkw')
SearchType eq LexisNexis.ServicesApi.SearchType'Boolean' and (PublicationType eq 'SW5kdXN0cnkgVHJhZGUgUHJlc3M' or PublicationType eq 'TmV3c3dpcmVzICYgUHJlc3MgUmVsZWFzZXM'or PublicationType eq 'TmV3c3BhcGVycw' or PublicationType eq 'TWFnYXppbmVzICYgSm91cm5hbHM') and Language eq LexisNexis.ServicesApi.Language'English' and GroupDuplicates eq LexisN